In [1]:
# !pip install --upgrade pip --quiet
# !pip install numpy --quiet
# !pip install pandas --quiet
# !pip install matplotlib --quiet
# !pip install scipy --quiet

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import timeit
import torch

In [3]:
# !wget -nc https://files.grouplens.org/datasets/movielens/ml-20m.zip
# !unzip -n ml-20m.zip

In [3]:
df = pd.read_csv('ml-20m/ratings.csv')
print(df.shape)
df.head()

(20000263, 4)


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [78]:
def make_csr(n,N):
    x = np.random.choice(N,n)
    y = np.random.choice(N,n)
    z = np.random.choice(5,n)+1
    print(z)
    return sparse.csr_matrix((z, (x,y)), shape=(N,N))
n_rows = 10
A = make_csr(25, n_rows)

[1 5 5 2 2 5 2 1 4 3 1 2 4 1 5 4 5 3 5 1 2 2 1 1 3]


In [79]:
A.toarray()

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 6, 0, 0, 0, 7],
       [0, 1, 0, 2, 0, 5, 0, 2, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 4, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 5, 0, 0],
       [0, 0, 1, 0, 3, 0, 0, 0, 0, 0],
       [5, 5, 0, 0, 3, 1, 2, 0, 0, 0],
       [0, 0, 4, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 5, 0, 0, 0]])

In [7]:
def method_1():
    u = np.random.randint(n_rows)
    user_row = A.getrow(u).indices
    user_data = A.getrow(u).data
    user_nnz = A.getrow(u).nnz
    for v in range(n_rows):
        count = A.getrow(v).nnz
        if count<10:
            continue
        row = A.getrow(v).indices
        data = A.getrow(v).data
        i, j = 0, 0
        a2, b2, ab, c = 0, 0, 0, 0
        while i<user_nnz and j<count:
            if user_row[i]==row[j]:
                a, b = user_data[i], data[j]
                a2+=(a*a); b2+=(b*b); ab+=(a*b); c+=1;
                i+=1; j+=1;
            elif user_row[i]<row[j]:
                i+=1;
            elif user_row[i]>row[j]:
                j+=1;

In [28]:
def method_2():
    ind = np.random.randint(n_rows)
    user = A.getrow(ind)
    user2 = user.power(2)
    for u in range(n_rows):
        row = A.getrow(u)
        ab = user.multiply(row)
        if ab.nnz<10:
            continue
        flag=(ab!=0)
        a2 = user2.multiply(flag)
        b2 = row.power(2).multiply(flag)
        sim = ab.sum()/np.sqrt(user2.multiply(flag).sum()*row.power(2).multiply(flag).sum())

In [12]:
a = A.getrow(0)
b = A.getrow(2)
ab = a.multiply(b)
flag = (ab!=0)
a2 = a.power(2).multiply(flag)
b2 = b.power(2).multiply(flag)

In [27]:
a.data, a.indices

(array([4, 3, 3, 1, 4, 2, 5, 3, 2, 2, 5, 1, 5, 4, 3, 1, 1, 4, 4, 3, 5, 4,
        4, 1, 1, 2, 3, 4, 2, 1, 4, 2, 4, 4, 4, 3, 3, 2, 1, 4, 3, 3, 3, 1,
        3, 2, 2, 3, 1, 5, 1, 3, 3, 5, 1, 3, 1, 1, 2, 5, 1, 1, 1, 2, 4, 2,
        3, 1, 5, 4, 3, 1, 4, 2, 4, 1, 4, 3, 2, 1, 2, 5, 2, 2, 5, 3, 1, 2,
        1, 4, 1, 5, 5, 5, 5, 4, 3, 1, 3, 3, 1, 2, 1, 4, 2, 3, 4, 3, 1, 5,
        5, 5, 2, 2, 4, 5, 3, 1, 3, 1, 4, 5, 5, 2, 1, 3, 4, 5, 2, 4, 3, 5,
        3, 3, 2, 4, 2, 3, 5, 4, 3, 5, 3, 3, 3, 3, 4, 4, 1, 1, 4, 4, 4, 5,
        2, 3, 2, 3, 5, 2, 3, 5, 4, 5, 1, 3, 2, 3, 1, 2, 1, 1, 3, 4, 1, 4,
        1, 2, 1, 1, 2, 1, 2, 2, 5, 1, 2, 4, 3, 1, 2, 5, 5, 3, 1, 1, 4, 1,
        1, 4, 3, 2, 2, 3, 3, 2, 2, 1, 5]),
 array([  315,   491,   507,   577,  1856,  2062,  3436,  3537,  4817,
         5093,  5195,  5485,  5737,  5893,  6081,  6186,  6462,  6687,
         7032,  7518,  7641,  7778,  8051,  9421,  9533,  9713, 10244,
        10285, 10648, 10709, 11879, 12266, 12441, 12625, 14093, 15294,
       

In [14]:
b.data, b.indices

(array([5, 4, 1, 3, 5, 1, 4, 5, 1, 1, 5, 2, 3, 4, 1, 1, 5, 1, 5, 4, 5, 1,
        2, 1, 4, 3, 1, 3, 2, 4, 4, 4, 4, 4, 3, 2, 3, 9, 3, 3, 3, 4, 1, 5,
        5, 2, 2, 3, 2, 1, 3, 5, 5, 4, 5, 5, 4, 1, 4, 3, 2, 2, 3, 2, 3, 2,
        1, 5, 4, 1, 4, 2, 3, 2, 5, 1, 4, 2, 3, 5, 3, 3, 5, 4, 4, 3, 4, 2,
        3, 3, 1, 4, 3, 5, 4, 5, 1, 3, 5, 2, 4, 1, 4, 4, 2, 2, 3, 2, 1, 4,
        1, 3, 3, 3, 3, 1, 2, 3, 3, 1, 2, 3, 5, 3, 4, 2, 2, 1, 4, 5, 5, 4,
        4, 4, 5, 4, 4, 4, 1, 5, 1, 3, 1, 4, 1, 4, 1, 3, 4, 3, 1, 2, 3, 5,
        3, 1, 3, 2, 3, 4, 3, 2, 4, 1, 5, 3, 5, 5, 2, 1, 2, 3, 4, 5, 5, 1,
        2, 2, 3, 3, 2, 3, 5, 4, 5, 1, 4, 4, 2, 2, 4, 5, 2, 5, 2, 4, 2, 1,
        3, 3, 1, 1, 2, 4, 4, 1, 4, 3, 1, 1]),
 array([  133,   148,   840,   871,  1099,  1669,  1703,  2083,  2101,
         2581,  2705,  3477,  3696,  5162,  5206,  5631,  6070,  6073,
         6104,  6276,  6315,  7239,  7845,  7966,  8079,  8163,  8765,
         9733, 10327, 10434, 11175, 13452, 13529, 13822, 13833, 13943,
    

In [15]:
ab.data, ab.indices

(array([], dtype=int64), array([], dtype=int32))

In [16]:
flag.data, flag.indices

(array([], dtype=bool), array([], dtype=int32))

In [17]:
a2.data, a2.indices

(array([], dtype=int64), array([], dtype=int32))

In [18]:
b2.data, b2.indices

(array([], dtype=int64), array([], dtype=int32))

In [19]:
ab.sum()/np.sqrt(a2.sum()*b2.sum())

/tmp/ipykernel_536/308889532.py:1: RuntimeWarning: invalid value encountered in scalar divide
  ab.sum()/np.sqrt(a2.sum()*b2.sum())


nan

In [20]:
a2.sum()

0

In [21]:
timeit.timeit(lambda : method_1(), number=10)

311.2983330099996

In [22]:
timeit.timeit(lambda : method_2(), number=10)

135.8911202220006

In [32]:
timeit.timeit(lambda : method_2(), number=10)

138.38761830500152

In [23]:
timeit.timeit(lambda : method_3(), number=10)

135.76580635199934

In [24]:
timeit.timeit(lambda : method_4(), number=10)

172.20552004799902

In [25]:
def test():
    r = np.random.randint(100000)
    for i in A.getrow(r).indices:
        x=i

In [26]:
%%timeit
test()

52.6 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
